# Support Vector Regression (SVR)

## Importing the libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import redshift_connector
import keyring

## Importing the dataset

In [24]:
### Importing via CSV
### dataset = pd.read_csv('Data.csv')

### Importing via Redshift
pwd = keyring.get_password("redshift-production.db.customink.com", "dan.caley")

# Connecting to redshift
# https://docs.aws.amazon.com/redshift/latest/mgmt/python-connect-examples.html#python-connect-query
conn = redshift_connector.connect(
    host='redshift-production.db.customink.com',
    database='cink',
    user='dan.caley',
    password= pwd
)

# Reading SQL File
open_file = open('sql_code.sql','r')
sql_file = open_file.read()
open_file.close()

# Running Query from sql file
dataset = pd.read_sql_query(sql_file, conn)

# Removing Binary headers
remove_binary = dataset.columns.astype(str).str.replace("'b",'')
dataset.columns = remove_binary


In [25]:
dataset.describe()

,days_deliverd,net_price,total_units,designs_prior_30,errors,zip_wealth,sales_bulk_following_365
count,70941.000000,70941.000000,70941.000000,70941.000000,70941.000000,7.094100e+04,70941.000000
mean,11.172312,598.665785,48.496032,2.536460,0.134732,2.407322e+09,1084.663598
std,6.040281,704.350811,92.846683,3.453797,0.341439,1.872485e+09,2468.901342
min,1.000000,0.200000,0.000000,0.000000,0.000000,0.000000e+00,1.530000
25%,8.000000,268.070000,13.000000,1.000000,0.000000,1.025876e+09,264.780000
50%,11.000000,399.040000,25.000000,1.000000,0.000000,2.038128e+09,493.000000
75%,13.000000,664.160000,50.000000,3.000000,0.000000,3.338458e+09,1061.000000
max,309.000000,31079.000000,5000.000000,137.000000,1.000000,1.608083e+10,90764.210000


In [26]:
# Check NA's
dataset.isna().sum()

customer_order_id            0
date_placed                  0
days_deliverd                0
net_price                    0
total_units                  0
designs_prior_30             0
errors                       0
zip_wealth                   0
segment_name_uber            0
segment_name_ultra           0
style_uber_category          0
style_category_utlra         0
uber_sales_channel_attr      0
uber_sales_channel_placed    0
sales_bulk_following_365     0
dtype: int64

## Splitting the dataset into the Training set and Test set

In [27]:
data = dataset[['days_deliverd','net_price','total_units','designs_prior_30','errors','zip_wealth','sales_bulk_following_365']]
### Breaking the Dependant and Independant Variables
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
y = y.reshape(len(y),1)

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feature Scaling

In [29]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train = sc_X.fit_transform(X_train)
y_train = sc_y.fit_transform(y_train)

## Training the SVR model on the Training set

In [30]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

C:\Users\dan.caley\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVR()

## Predicting the Test set results

In [31]:
y_pred = sc_y.inverse_transform(regressor.predict(sc_X.transform(X_test)))
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[ 995.97 1940.66]
 [ 778.53 2113.72]
 [ 474.95  262.8 ]
 ...
 [ 552.54  390.21]
 [ 939.56 4423.17]
 [ 426.32  212.24]]


## Evaluating the Model Performance

In [32]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.021730530175815277